# Transforms

- 译文：https://pytorch.apachecn.org/2.0/tutorials/beginner/basics/transforms_tutorial
- 原文：https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html

## 概览

- 数据常不是训练所需的最终格式，需通过 transforms 将数据转换为模型可用的格式。
- TorchVision 的各个 Dataset 接受两个可选参数：`transform`（用于变换特征）和 `target_transform`（用于变换标签），两者都接受可调用对象。
- `torchvision.transforms` 模块提供了多种常用变换，且可组合使用。

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt

In [ ]:
ds = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

print('Loaded dataset with transforms; samples:', len(ds))
# 查看一个样本（特征为 tensor，标签为 one-hot tensor）
img, label = ds[0]
print('img type/shape:', type(img), img.shape)
print('label (one-hot):', label)

# 可视化（如果运行环境支持）
plt.imshow(img.squeeze(), cmap='gray')
plt.title(f'one-hot label sum: {label.sum()}')
plt.axis('off')
plt.show()

### ToTensor()

- `ToTensor` 将 PIL 图像或 NumPy ndarray 转换为 `torch.FloatTensor`，并把像素值缩放到 `[0., 1.]`。这通常是将图像输入神经网络前的第一步变换。

### Lambda Transforms（标签变换示例）

- `Lambda` 接受任意用户函数用于对样本或标签进行自定义变换。下面示例将整数标签转换为 one-hot 编码的 tensor：

In [ ]:
target_transform = Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

- 解释：先创建长度为 10 的零 tensor，再使用 `scatter_` 在索引 `y` 位置写入 1，从而得到 one-hot 向量。注意 `scatter_` 是就地操作，会修改目标 tensor。

## 注意事项与参考

- 常用变换还包括 `Normalize`（归一化）、`Resize`、`RandomCrop`、`RandomHorizontalFlip` 等，可在 `torchvision.transforms` 中找到。可通过 `Compose` 串联多个变换。
- 注意避免在训练时对验证/测试数据应用随机变换（如随机裁剪、翻转等）。